In [1]:
!git clone https://github.com/princeton-nlp/MeZO.git

Cloning into 'MeZO'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 173 (delta 45), reused 30 (delta 30), pack-reused 98 (from 1)
Receiving objects: 100% (173/173), 432.68 KiB | 8.16 MiB/s, done.
Resolving deltas: 100% (88/88), done.


In [1]:
# Install the latest compatible PyTorch version with CUDA 11.8
!pip install torch --index-url https://download.pytorch.org/whl/cu118
!pip install transformers==4.28.1
!sudo apt-get install jq
!pip install loralib
!pip install nvidia-ml-py3
!pip install datasets

Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package jq
  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19171 sha256=aa2527e2dfe27c4aa305e4fdbcbbce39a6316b59dae897efa420c3ff09c52c33
  Stored in directory: /home/javad/.cache/pip/wheels/47/50/9e/29dc79037d74c3c1bb4a8661fb608e8674b7e4260d6a3f8f51
Successfully built nvidia-ml-py3


In [2]:
!sudo apt-get update
!sudo apt-get install jq -y

Get:1 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 Packages [266 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy/main amd64 Packages [1792 kB]     
Get:7 http://archive.ubuntu.com/ubuntu jammy/restricted amd64 Packages [164 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy/universe amd64 Packages [17.5 MB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1243 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4000 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1542 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3099 kB]
Get:13 http://security.ubuntu.com/ub

In [5]:
!which jq

/usr/bin/jq


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib.backends.backend_pdf import PdfPages
import time
import csv
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetPowerUsage, nvmlShutdown
import os
import subprocess
import torch
from pynvml import *
from threading import Thread
import re
import json
import numpy as np
from pathlib import Path
from transformers import RobertaForSequenceClassification, RobertaTokenizer, Trainer, TrainingArguments, AutoModel
from datasets import load_dataset
import torch.nn.utils.prune as prune
from pathlib import Path
import matplotlib.pyplot as plt
import random
import matplotlib.colors as mcolors
import subprocess
import glob
from dataclasses import dataclass
from datetime import datetime
import gc

In [ ]:
%cd medium_models/data
!bash download_dataset.sh

In [ ]:
%cd medium_models
!python tools/generate_k_shot_data.py --mode k-shot-1k-test --k 128 --task SST-2 --seed 42

In [ ]:
def split_train_tsv(file_path, num_parts=10, seed=42):
    random.seed(seed)

    with open(file_path, "r") as f:
        lines = f.readlines()

    header = lines[0]
    data_lines = lines[1:]

    # Group by label
    label_dict = {}
    for line in data_lines:
        label = line.strip().split('\t')[-1]
        if label not in label_dict:
            label_dict[label] = []
        label_dict[label].append(line)

    # Ensure we have only two labels
    assert len(label_dict) == 2, f"Expected binary classification, found labels: {list(label_dict.keys())}"

    # Shuffle each label group
    for label in label_dict:
        random.shuffle(label_dict[label])

    # Calculate min samples per label
    min_len = min(len(label_dict[label]) for label in label_dict)
    part_len = min_len // num_parts

    for i in range(num_parts):
        part_lines = []
        for label in sorted(label_dict.keys()):
            start_idx = i * part_len
            end_idx = (i + 1) * part_len
            part_lines.extend(label_dict[label][start_idx:end_idx])

        # Shuffle mixed part lines before saving
        random.shuffle(part_lines)

        part_path = file_path.replace("train.tsv", f"train_part{i+1}.tsv")
        with open(part_path, "w") as out:
            out.write(header)
            for line in part_lines:
                out.write(line)

        print(f"Saved: {part_path}")

split_train_tsv(f"data/k-shot-1k-test/SST-2/1280-42/train.tsv")


In [7]:
def evaluate_model(model_dir):
    tokenizer = RobertaTokenizer.from_pretrained(model_dir)
    model = RobertaModelForPromptFinetuning.from_pretrained(model_dir)
    model.model_args = DynamicTrainingArguments()
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    test_samples = [
        {"text": "This movie is great! It was", "label": "positive"},
        {"text": "I hated it. It was", "label": "negative"}
    ]
    label_words = {"positive": "Ġgreat", "negative": "Ġterrible"}
    label_ids = {label: tokenizer.convert_tokens_to_ids(word) for label, word in label_words.items()}

    correct = 0
    with torch.no_grad():
        for sample in test_samples:
            text_with_mask = f"{sample['text']} <mask>."
            inputs = tokenizer(text_with_mask, return_tensors="pt", truncation=True, padding=True, max_length=128)
            input_ids = inputs["input_ids"].to(device)
            attention_mask = inputs["attention_mask"].to(device)

            print(f"Text: {text_with_mask}")
            print(f"Input IDs: {input_ids}")
            print(f"Attention Mask: {attention_mask}")
            mask_pos = torch.where(input_ids[0] == tokenizer.mask_token_id)[0].item()
            print(f"Mask Position: {mask_pos}")

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, mask_pos=torch.tensor([mask_pos]).to(device))
            logits = outputs[0]
            print(f"Logits Shape: {logits.shape}")

            if len(logits.shape) == 2 and logits.shape[1] == 2:
                roberta_outputs = model.roberta(input_ids=input_ids, attention_mask=attention_mask)
                hidden_states = roberta_outputs[0]
                if hasattr(model, 'lm_head'):
                    logits = model.lm_head(hidden_states)
                else:
                    logits = model.classifier(hidden_states)
                    if logits.shape[-1] == 2:
                        raise ValueError("Model outputs classification logits instead of token logits. Missing LM head?")
                print(f"Corrected Logits Shape: {logits.shape}")

            logits_at_mask = logits[0, mask_pos]
            prob_great = logits_at_mask[label_ids["positive"]].item()
            prob_terrible = logits_at_mask[label_ids["negative"]].item()
            pred_label = "positive" if prob_great > prob_terrible else "negative"

            print(f"Predicted: {pred_label}, Expected: {sample['label']}")
            print(f"Logits - great: {prob_great}, terrible: {prob_terrible}")
            correct += (pred_label == sample["label"])

    accuracy = correct / len(test_samples) * 100
    print(f"Evaluation accuracy for {model_dir}: {accuracy}%")
    return accuracy

def extract_metrics_from_log(log_file):
    eval_loss = None
    eval_acc = None
    timestamp = None
    
    with open(log_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            if "eval_loss =" in line:
                timestamp = line.split(" - ")[0]
                eval_loss = float(line.split("eval_loss =")[1].strip())
            elif "eval_acc =" in line:
                eval_acc = float(line.split("eval_acc =")[1].strip())
    
    return timestamp, eval_loss, eval_acc

def save_client_metrics(client_id, round_num, log_file):
    timestamp, eval_loss, eval_acc = extract_metrics_from_log(log_file)
    if eval_loss is not None and eval_acc is not None and timestamp is not None:
        filename = f"client_{client_id}.txt"
        with open(filename, 'a') as f:
            f.write(f"{timestamp} - INFO - __main__ -     eval_loss = {eval_loss}\n")
            f.write(f"{timestamp} - INFO - __main__ -     eval_acc = {eval_acc}\n")

In [ ]:
@dataclass
class DynamicTrainingArguments:
    sfc: bool = False
    use_task_word: bool = False
    num_labels: int = 2
    binary_classification: bool = True
    ub: float = 1.0
    lb: float = 0.0

try:
    from src.models import RobertaModelForPromptFinetuning
except ImportError:
    from transformers import RobertaForSequenceClassification as RobertaModelForPromptFinetuning
    print("Warning: Using fallback model.")

def main():
    num_rounds = 5
    clients = 10
    parent_folder = "output_model"
    global_model_output_dir = "saved_model"
    k = 128
    seed = 42
    
    #bp as 0, bp free as 1
    status = [0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
    
    # Clear existing client files
    for client_id in range(1, clients + 1):
        filename = f"client_{client_id}.txt"
        if os.path.exists(filename):
            os.remove(filename)

    for round_num in range(1, num_rounds + 1):
        print(f"Starting Round {round_num}")
        averaged_state_dict = {}

        for client_id in range(1, clients + 1):
            print(f"------------------------------------Executing command for Client {client_id} in Round {round_num}:")
            directory = f"data/k-shot-1k-test/SST-2/{k}-{seed}/"
            cached_files = glob.glob(os.path.join(directory, "cached*"))
            for file_path in cached_files:
                try:
                    os.remove(file_path)
                    print(f"Deleted: {file_path}")
                except Exception as e:
                    print(f"Error deleting {file_path}: {e}")

            command = f"mv data/k-shot-1k-test/SST-2/{k}-{seed}/train_part{client_id}.tsv data/k-shot-1k-test/SST-2/{k}-{seed}/train.tsv"
            os.system(command)

            log_file = f"mezo_round_{round_num}_client_{client_id}.log"
            if status[client_id-1] == 0:
                print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>USING BP")
                if round_num == 1:
                    command = f"WANDB_MODE=offline TASK=SST-2 K={k} SEED={seed} BS=64 LR=1e-6 STEP=10 EVAL_STEP=10 MODEL=roberta-large bash mezo.sh"
                    os.system(command)
                else:
                    command = f"WANDB_MODE=offline TASK=SST-2 K={k} SEED={seed} BS=64 LR=1e-6 STEP=10 EVAL_STEP=10 MODEL={global_model_output_dir} bash mezo.sh"
                    os.system(command)
            
            if status[client_id-1] == 1:
                print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>USING BP Free")
                if round_num == 1:
                    command = f"WANDB_MODE=offline TASK=SST-2 K={k} SEED={seed} BS=64 LR=1e-6 STEP=10 EVAL_STEP=10 MODEL=roberta-large bash mezo.sh"
                    os.system(command)
                else:
                    command = f"WANDB_MODE=offline TASK=SST-2 K={k} SEED={seed} BS=64 LR=1e-6 STEP=10 EVAL_STEP=10 MODEL={global_model_output_dir} bash mezo.sh"
                    os.system(command)
            
            with open(log_file, "w") as f:
                process = subprocess.run(command, shell=True, stdout=f, stderr=subprocess.STDOUT, text=True)
            # with open(log_file, "r") as f:
            #     print(f"MeZO stdout:\n{f.read()}")
            if process.returncode != 0:
                raise RuntimeError("MeZO execution failed")

            # Save metrics from log
            save_client_metrics(client_id, round_num, log_file)

            command = f"mv data/k-shot-1k-test/SST-2/{k}-{seed}/train.tsv data/k-shot-1k-test/SST-2/{k}-{seed}train_part/{client_id}.tsv"
            os.system(command)

            model = RobertaModelForPromptFinetuning.from_pretrained(parent_folder)
            model.model_args = DynamicTrainingArguments()
            tokenizer = RobertaTokenizer.from_pretrained(parent_folder)
            state_dict = model.state_dict()

            if not averaged_state_dict:
                averaged_state_dict = {k: v.clone() for k, v in state_dict.items()}
            else:
                for key in state_dict.keys():
                    if state_dict[key].is_floating_point():
                        averaged_state_dict[key] += state_dict[key]
                    else:
                        averaged_state_dict[key] = state_dict[key].clone()

            os.system(f"rm -rf {parent_folder}")

        print("Averaging weights")
        for key in averaged_state_dict.keys():
            if averaged_state_dict[key].is_floating_point():
                averaged_state_dict[key] /= clients

        if round_num == 1:
            global_model = RobertaModelForPromptFinetuning.from_pretrained('roberta-large')
        else:
            global_model = RobertaModelForPromptFinetuning.from_pretrained(global_model_output_dir)
        global_model.model_args = DynamicTrainingArguments()
        global_model.load_state_dict(averaged_state_dict)
        global_model.save_pretrained(global_model_output_dir)
        tokenizer.save_pretrained(global_model_output_dir)
        print(f"Global model saved to {global_model_output_dir}")

        evaluate_model(global_model_output_dir)
        
        torch.cuda.empty_cache()  # Clear the GPU cache
        gc.collect() 

    print("All rounds completed.")

if __name__ == "__main__":
    main()